{{title_its_nb4}}

{{intro}}
In the previous notebook, we walked through initial steps to read and organize a large raster dataset, and to understand it in the context of spatial areas of interest represented by vector data. 

The previous notebook mainly focused on high-level processing operations:   

1) Reading a multi-dimensional dataset,   
2) Re-organizing along a given dimension,   
3) Reducing a large raster object to smaller areas of interest represented by vector data, and   
4) Strategies for completing these steps with larger-than-memory datasets.   

In this notebook, we will continue performing initial data inspection and exploratory analysis but this time, focused on velocity data clipped to an individual glacier. We demonstrate Xarray functionality for common computations and visualizations.

::::{tab-set}
:::{tab-item} Outline

(content:Section_A)=
**[A. Data exploration](#a-data-exploration)**
- {{a1_its_nb4}}
- {{a2_its_nb4}}
- {{a3_its_nb4}}

(content:Section_B)=
**[B. Comparing different satellites](#b-comparing-different-satellites)**
- {{b1_its_nb4}}
- {{b2_its_nb4}}

(content:Section_C)=
**[C. Examine velocity variability](#c-examine-velocity-variability)**
- {{c1_its_nb4}}
- {{c2_its_nb4}}
- {{c3_its_nb4}}

(content:section_D)=
**[D. Computations  along time dimension](#d-computations--along-time-dimension)**
- {{d1_its_nb4}}
- {{d2_its_nb4}}
:::

:::{tab-item} Learning Goals

{{concepts}}
- Examining metadata, interpreting physical observable in the context of available metadata,
- Sub-setting and visualizing raster datasets
- 'Split-apply-combine' workflows,
- Calculating summary statistics across given dimensions of a dataset,
- Performing reductions across multi-dimensional datasets.

{{techniques}}
- Managing groups of Xarray objects with `Xarray.DataTree`,  
- Grouped computations with `GroupBy`,  
- Using matplotlib to visualize raster and vector data with satellite imagery basemaps,   
- Combining Xarray with statistical packages like [SciPy](https://scipy.org/).

:::
::::

{{break}}

Expand the next cell to see specific packages used in this notebook and relevant system and version information. 

In [ ]:
%xmode minimal
import warnings

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import scipy.stats
import xarray as xr

warnings.simplefilter(action="ignore", category=FutureWarning)

Like in the previous notebook, we will write some objects to disk. Create the same variable to store the path to the root directory for this tutorial.

In [ ]:
cwd = pathlib.Path.cwd()
tutorial1_dir = pathlib.Path(cwd).parent

{{break}}

## A. Data exploration

### {{a1_its_nb4}}

In [ ]:
# Read raster
single_glacier_raster = xr.open_zarr("../data/raster_data/single_glacier_itslive.zarr", decode_coords="all")
# Read vector
single_glacier_vector = gpd.read_file("../data/vector_data/single_glacier_vec.json")

In [ ]:
single_glacier_raster.nbytes / 1e9

In [ ]:
np.unique(single_glacier_raster.satellite_img1)

In [ ]:
np.unique(single_glacier_raster.satellite_img2)

The above code cells show us that this dataset contains observations from Sentinel 1 & 2 and Landsat 4,5,6,7,8 & 9 satellite sensors. The dataset is 3.3 GB. 

Next, we want to perform computations that require us to load this object into memory. To do this, we use the [dask .compute()](https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.compute.html) method, which turns a 'lazy' object into an in-memory object. If you try to run compute on too large of an object, your computer may run out of RAM and the kernel being used in this python session will die (if this happens, click 'restart kernel' from the kernel drop down menu above).  

In [ ]:
single_glacier_raster = single_glacier_raster.compute()

Now, if you expand the data object to look at the variables, you will see that they no long hold `dask.array` objects.

In [ ]:
single_glacier_raster

As a quick sanity check, we'll convince ourselves that the clipping operation in the previous [notebook](initial_velocity_data_inspection.ipynb#A.-Crop-ITS_LIVE-granule-single-glacier-outline) worked correctly. We also show that we can plot both Xarray raster data and GeoPandas vector data overlaid on the same plot with a satellite image basemap as a background.

A few notes about the following figure:
- `single_glacier_raster` is a 3-dimensional object. We want to plot it in 2-d space with the RGI glacier outline. So, we perform a reduction (in this case, compute the mean), in order to reduce the dataset from 3-d to 2-d (Another option would be to select a single time step).
- We could make this plot with a white background, but it is also nice to be able to add a base map to the image. Here, we'll use [contextily](https://contextily.readthedocs.io/en/latest/) to do so. This will require converting the coordinate reference system (CRS) of both objects to the Web Mercator projection (EPSG:3857).

In [ ]:
# Check that CRS of vector and raster data are the same
assert single_glacier_raster.rio.crs == single_glacier_vector.crs

In [ ]:
single_glacier_raster = single_glacier_raster.rio.write_crs(single_glacier_raster.attrs["projection"])

In [ ]:
# Check that CRS of vector and raster data are the same
assert single_glacier_raster.rio.crs == single_glacier_vector.crs

In [ ]:
# Reproject both objects to web mercator
single_glacier_vector_web = single_glacier_vector.to_crs("EPSG:3857")
single_glacier_raster_web = single_glacier_raster.rio.reproject("EPSG:3857")

fig, ax = plt.subplots(figsize=(8, 5))

# Plot objects
single_glacier_raster_web.v.mean(dim="mid_date").plot(ax=ax, cmap="viridis", alpha=0.75, add_colorbar=True)
single_glacier_vector_web.plot(ax=ax, facecolor="None", edgecolor="red", alpha=0.75)
# Add basemap
cx.add_basemap(ax, crs=single_glacier_vector_web.crs, source=cx.providers.Esri.WorldImagery)

We sorted along the time dimension in the previous notebook, so it should be in chronological order.

In [ ]:
single_glacier_raster.mid_date[:6].data

### {{a2_its_nb4}}

A wide variety of forces can impact both satellite imagery and the ability of ITS_LIVE's feature tracking algorithm to extract velocity estimates from satellite image pairs. For these reasons, there are at times both gaps in coverage and ranges in the estimated error associated with different observations. The following section will demonstrate how to calculate and visualize coverage of the dataset over time. Part 2 will include a discussion of uncertainty and error estimates

When first investigating a dataset, it is helpful to be able to scan/quickly visualize coverage along a given dimension. To create the data needed for such a visualization, we first need a mask that will tell us all possible 'valid' pixels; in other words, we need to differentiate between pixels in our 2-d rectangular array that represent ice v. non-ice. Then, for every time step, we can calculate the portion of possible ice pixels that contain an estimated velocity value.

In [ ]:
# calculate number of valid pixels
valid_pixels = single_glacier_raster.v.count(dim=["x", "y"])
# calculate max. number of valid pixels
valid_pixels_max = single_glacier_raster.v.notnull().any("mid_date").sum(["x", "y"])
# add cov proportion to dataset as variable
single_glacier_raster["cov"] = valid_pixels / valid_pixels_max

Now we can visualize coverage over time:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

# Plot object
single_glacier_raster["cov"].plot(ax=ax, linestyle="None", marker="x", alpha=0.75)

# Specify axes labels and title
fig.suptitle("Velocity data coverage ovver time", fontsize=16)
ax.set_ylabel("Coverage (proportion)", x=-0.05, fontsize=12)
ax.set_xlabel("Date", fontsize=12);

### {{a3_its_nb4}}

In this dataset, we have a dense time series of velocity observations for a given glacier (~48,000 observations from 1986-2024). However, we know that the ability of satellite imagery pairs to capture ice displacement (and by extension, velocity), can be impacted by conditions such as cloud-cover, which obscure Earth's surface from optical sensors. ITS_LIVE is a multi-sensor ice velocity dataset, meaning that it is composed of ice velocity observations derived from a number of satellites, which include both optical and Synthetic Aperture Radar (SAR) imagery. Currently, Sentinel-1 is the only SAR sensor included in ITS_LIVE, all others are optical. 

While optical imagery requires solar illumination and can be impacted by cloud cover, Sentinel-1 is an active remote sensing technique and images in a longer wavelength (C-band, ~ 5 cm). This means that Sentinel-1 imagery does not require solar illumination, and can penetrate through cloud cover. Because of these sensors differing sensitivity to Earth's surface conditions, there can sometimes be discrepancies in velocity data observed from different sensors. 

```{note}
There are many great resources available for understanding the principles of SAR and working with SAR imagery. Check out the SAR section of the [tutorial data](../../background/4_tutorial_data.md) for more detail and links to additional resources.
```

Let's first look at what sensors are represented in the time series:

In [ ]:
sensors = np.unique(single_glacier_raster.satellite_img1.values).tolist()
sensors

To extract observations from a single satellite sensor, we will use Xarray indexing and selection methods such as `.where()` and `.sel()`. The following cells demonstrate different selection approaches and a brief discussion of the pros and cons of each when working with large and/ or sparse datasets.

#### Landsat 8
First, looking at velocity observations from Landsat8 data only:

In [ ]:
%%time
l8_data = single_glacier_raster.where(single_glacier_raster["satellite_img1"] == "8", drop=True)
l8_data

Another approach: using `.sel()`:

In [ ]:
%%time
l8_condition = single_glacier_raster.satellite_img1 == "8"
l8_data_alt = single_glacier_raster.sel(mid_date=l8_condition)
l8_data_alt

The approach using `.sel()` takes much less time than `.where()`. This is because `.sel()` queries the dataset using `mid_date` index. Xarray dimensions have associated `Index` objects, which are built on Pandas Indexes. They are very powerful for quickly and efficiently querying large datasets. In contrast, `.where()` must check each `satellite_img` data variable against the specified condition ('8'), which is not as efficient. 

#### What about a sensor with multiple identifiers?

For Landsat8 observations, we only needed to identify elements of the dataset where the `satellite_img1` variable matched a unique identifier, '8'. Sentinel-1 is a two-satellite constellation, meaning that it has sensors on board multiple satellites. For this, we need to select all observations where `satellite_img1` matches a list of possible values.

In [ ]:
%%time
s1_condition = single_glacier_raster.satellite_img1.isin(["1A", "1B"])
s1_data = single_glacier_raster.sel(mid_date=s1_condition)
s1_data

## B. Comparing different satellites

We've seen how to use Xarray's labeled dimensions and indexing and selection capabilities to subset the dataset based on its metadata. What if we wanted to use those subsets, for example to visualize data or perform computations for each subset group? We could do this manually, but it would be quite time-consuming and involve repeating the same steps for each group. Luckily, there are a few different ways that we can approach this without writing a lot of duplicate code. 

Below, we show two different approaches to split the dataset into subset based on a metadata condition, perform a calculation on each subset and then visualize the results for each subset side-by-side. 

Start by making a dict of each sensor and its identifying string(s):

In [ ]:
sensor_conditions = {
    "Landsat 4": ["4"],
    "Landsat 5": ["5"],
    "Landsat 7": ["7"],
    "Landsat 8": ["8"],
    "Landsat 9": ["9"],
    "Sentinel 1": ["1A", "1B"],
    "Sentinel 2": ["2A", "2B"],
}

### {{b1_its_nb4}}

Rather than go through the above steps for each sensor, we write a function that subsets the dataset by sensor, returning a dict of Xarray datasets holding velocity time series for each sensor.

In [ ]:
def separate_ds_by_sensor(ds, sensor_conditions):
    # Make empty lists to hold sensor IDs and subsetted datasets
    keys_ls, vals_ls = [], []

    # Iterate through each sensor in dict
    for sensor in sensor_conditions.keys():
        condition = ds.satellite_img1.isin(sensor_conditions[sensor])

        # Use .sel to subset data based on sensor
        sensor_data = ds.sel(mid_date=condition)

        keys_ls.append(f"{sensor}")
        vals_ls.append(sensor_data)

    # Return dict of sensor IDs and subsetted datasets
    ds_dict = dict(zip(keys_ls, vals_ls))
    return ds_dict

In [ ]:
sensor_ds_dict = separate_ds_by_sensor(single_glacier_raster, sensor_conditions)
print(sensor_ds_dict.keys())

Take a look at one subset to see that it matches the one we made manually above:

In [ ]:
sensor_ds_dict["Landsat 8"]

Now that we've separated the dataset into subset, we'd like a way to work with those subsets as a group, rather than applying operations to each one. For this, we can use a new structure within the Xarray data model, `xr.DataTree`, which facilitates working with collections of Xarray data objects. 

The use-case in this section is that we would like to efficiently compute the mean along the time dimension of the ITS_LIVE dataset for each satellite sensor, and then visualize the results.

Before the implementation of Xarray DataTree, we would need to either perform the sequence of operations on each sensor-specific dataset individually, or, create a dict of the sensor specific datasets, write a function to perform certain operations and either update the dictionary or create another to hold the results; both of these options are quite clunky and inefficient. 

We can create an `xr.DataTree` object by using the `from_dict()` method:


In [ ]:
sensor_ds_tree = xr.DataTree.from_dict(sensor_ds_dict)

In [ ]:
sensor_ds_tree

The DataTree has a parent node and groups (shown above) that contain individual `xr.Datasets`:

In [ ]:
sensor_ds_tree["Landsat 8"].ds

If we want to perform a set of operations on every node of the Datatree, we can define a function to pass to `xr.DataTree.map_over_datasets()`:

In [ ]:
def calc_temporal_mean(ds):
    """I'm a function that calculates the temporal mean of a dataset with (x,y,mid_date) dimensions.
    I return a new dataset with (x,y,sensor) dimensions"""

    # Skip parent node
    if len(ds.data_vars) == 0:
        return None

    else:
        # Calc mean
        tmean = ds.mean(dim="mid_date")
        # Add a sensor dimension -- this will be used for combining them back together later
        sensor = np.unique(ds.satellite_img1.data)
        if len(sensor) > 1:
            sensor = [sensor[0]]
        # Expand dims to add sensor
        tmean = tmean.expand_dims({"sensor": sensor})
        return tmean

In [ ]:
temp_mean_tree = sensor_ds_tree.map_over_datasets(calc_temporal_mean)

Now, we can take just the descendent nodes of this Datatree:

In [ ]:
child_tree = temp_mean_tree.descendants

and concatenate them into a single `xr.Dataset` along the `'sensor'` dimension we created above:

In [ ]:
sensor_mean_ds = xr.concat([child_tree[i].ds for i in range(len(child_tree))], dim="sensor")

In [ ]:
sensor_mean_ds

Finally, we can use Xarray's `FacetGrid` plotting to visualize them side-by-side:

In [ ]:
def calc_v_magnitude(ds):
    """I'm a function that calculates the magnitude of a velocity displacement vector given velocity component vectors.
    I return the same dataset object with a new variable called 'vmag'"""

    ds["vmag"] = np.sqrt(ds["vx"] ** 2 + ds["vy"] ** 2)
    return ds


# First, calculate magnitude of velocity from the temporal mean of the component vectors
sensor_mean_ds = calc_v_magnitude(sensor_mean_ds)

a = sensor_mean_ds["vmag"].plot(col="sensor", col_wrap=4, cbar_kwargs={"label": "Meters / year"})
a.fig.suptitle("Temporal mean of velocity magnitude", fontsize=16, y=1.05)
a.fig.supylabel("Y-coordinate of projection (meters)", fontsize=12, x=-0.02)
a.fig.supxlabel("X-coordinate of projection (meters)", fontsize=12, y=-0.05)

# remove individual axes labesl
for i in range(len(a.axs[0])):
    a.axs[0][i].set_ylabel(None)
    a.axs[0][i].set_xlabel(None)

for i in range(len(a.axs[1])):
    a.axs[1][i].set_ylabel(None)
    a.axs[1][i].set_xlabel(None)
    a.axs[1][i].tick_params(axis="x", labelrotation=45)

It's important to keep in mind that in addition to having different spectral properties and imaging resolutions, the sensors included in the ITS_LIVE dataset have been active during different, and sometimes overlapping periods of time. There is additional discussion of inter-sensor bias in the ITS_LIVE Known Issues [documentation](http://its-live-data.jpl.nasa.gov.s3.amazonaws.com/documentation/ITS_LIVE-Regional-Glacier-and-Ice-Sheet-Surface-Velocities-Known-Issues.pdf).

### {{b2_its_nb4}}


The previous section is a great example of a split-apply-combine workflow, where we split the dataset up in to multiple datasets by sensor, applied the `mean` computation, and then combined the results back. This is the GroupBy paradigm.

An alternate approach to the above figure would be to first use the `sensor_conditions` mapping to create a variable to group by:

In [ ]:
sensor = single_glacier_raster.satellite_img1
sensor.name = "sensor"
for label, values in sensor_conditions.items():
    sensor = xr.where(sensor.isin(values), label, sensor)
sensor

Now group the data by sensor

In [ ]:
grouped = single_glacier_raster.groupby(sensor)

Apply the mean reduction to each group

In [ ]:
sensor_means = grouped.mean()

We now have a 3D Dataset:

In [ ]:
sensor_means

Now we will make the same figure as earlier

In [ ]:
sensor_means = calc_v_magnitude(sensor_means)

a = sensor_means["vmag"].plot(col="sensor", col_wrap=4, cbar_kwargs={"label": "Meters / year"})
a.fig.suptitle("Temporal mean of velocity magnitude", fontsize=16, y=1.05)
a.fig.supylabel("Y-coordinate of projection (meters)", fontsize=12, x=-0.02)
a.fig.supxlabel("X-coordinate of projection (meters)", fontsize=12, y=-0.05)

# remove individual axes labesl
for i in range(len(a.axs[0])):
    a.axs[0][i].set_ylabel(None)
    a.axs[0][i].set_xlabel(None)

for i in range(len(a.axs[1])):
    a.axs[1][i].set_ylabel(None)
    a.axs[1][i].set_xlabel(None)
    a.axs[1][i].tick_params(axis="x", labelrotation=45)

#### Choosing an approach

The two paradigms, hierarchical Datatree and groupby, are equivalent ways of solving the problem. Choose the approach that makes most sense for your analysis. In this case our analysis is a simple operation (`mean`) and the result is a regular 3D array, so the groupby approach is less complex. For more complicated datasets, for example those where each sensor is a separate cube on a different grid, the Datatree approach is more appropriate and ergonomic.

## C. Examine velocity variability

### {{c1_its_nb4}}

First, we plot histogram of the `v`, `vx` and `vy` variables to examine their distributions. To construct these plots, we use a combination of xarray plotting functionality and matplotlib object-oriented plotting. In addition, we use [`xr.reduce()`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.reduce.html) and [`scipy.stats.skew()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html) to calculate the skew of each variable (inset in each sub-plot).

To make things easier, write a function that calculates and hold summary statistics for each variable in a dictionary:

In [ ]:
def calc_summary_stats(ds: xr.Dataset, variable: str):
    """I'm a function that calculates summary statistics for a given data variable and returns them as a dict to be used in a plot"""

    skew = ds[f"{variable}"].reduce(func=scipy.stats.skew, nan_policy="omit", dim=["x", "y", "mid_date"]).data
    mean = ds[f"{variable}"].mean(dim=["x", "y", "mid_date"], skipna=True).data
    median = ds[f"{variable}"].median(dim=["x", "y", "mid_date"], skipna=True).data

    stats_dict = {"skew": skew, "mean": mean, "median": median}
    return stats_dict

In [ ]:
stats_vy = calc_summary_stats(single_glacier_raster, "vy")
stats_vx = calc_summary_stats(single_glacier_raster, "vx")
stats_v = calc_summary_stats(single_glacier_raster, "v")

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
# VY
hist_y = single_glacier_raster.vy.plot.hist(ax=axs[0], bins=100)
cumulative_y = np.cumsum(hist_y[0])
axs[0].plot(hist_y[1][1:], cumulative_y, color="orange", linestyle="-", alpha=0.5)
# VY stats text
axs[0].text(x=-2000, y=2e6, s=f"Skew: {stats_vy['skew']:.3f}", fontsize=12, color="black")
axs[0].text(x=-2000, y=1.5e6, s=f"Mean: {stats_vy['mean']:.3f}", fontsize=12, color="black")
axs[0].text(x=-2000, y=1e6, s=f"Median: {stats_vy['median']:.3f}", fontsize=12, color="black")

# VX
hist_x = single_glacier_raster.vx.plot.hist(ax=axs[1], bins=100)
cumulative_x = np.cumsum(hist_x[0])
axs[1].plot(hist_x[1][1:], cumulative_x, color="orange", linestyle="-", alpha=0.5)
# VX stats text
axs[1].text(x=-2000, y=2e6, s=f"Skew: {stats_vx['skew']:.3f}", fontsize=12, color="black")
axs[1].text(x=-2000, y=1.5e6, s=f"Mean: {stats_vx['mean']:.3f}", fontsize=12, color="black")
axs[1].text(x=-2000, y=1e6, s=f"Median: {stats_vx['median']:.3f}", fontsize=12, color="black")

# V
hist_v = single_glacier_raster.v.plot.hist(ax=axs[2], bins=100)
cumulative_v = np.cumsum(hist_v[0])
axs[2].plot(hist_v[1][1:], cumulative_v, color="orange", linestyle="-", alpha=0.5)
# V stats text
axs[2].text(x=2000, y=2e6, s=f"Skew: {stats_v['skew']:.3f}", fontsize=12, color="black")
axs[2].text(x=2000, y=1.5e6, s=f"Mean: {stats_v['mean']:.3f}", fontsize=12, color="black")
axs[2].text(x=2000, y=1e6, s=f"Median: {stats_v['median']:.3f}", fontsize=12, color="black")

# Formatting and labeling
axs[0].set_title("VY")
axs[1].set_title("VX")
axs[2].set_title("V)")

for i in range(len(axs)):
    axs[i].set_xlabel(None)
    axs[i].set_ylabel(None)

fig.supylabel("# Observations", x=0.08, fontsize=12)
fig.supxlabel("Meters / year", fontsize=12)
fig.suptitle(
    "Histogram (blue) and cumulative distribution function (orange) of velocity components and magnitude",
    fontsize=16,
    y=1.05,
);

The histograms and summary statistics show that `vx` and `vy` distributions are relatively Gaussian, while `v` is positively skewed and [Rician](https://en.wikipedia.org/wiki/Rice_distribution). This is due to the non-linear relationship between component and displacement vectors. In datasets such as this one where the signal to noise ratio can be low, calculating velocity magnitude on smoothed or averaged component vectors can help to suppress noise (for a bit more detail, refer to this [comment](https://github.com/e-marshall/itslive/issues/26)). For this reason, we will usually calculate velocity magnitude after the dataset has been reduced over space or time dimensions.

### {{c2_its_nb4}}

Now that we have a greater understanding of the importance of velocity component variability in shaping our understandings of velocity variability, let's examine these variables as well as the estimated error provided in the dataset by reducing the dataset along the temporal dimension so that we can visualize the data along x and y dimensions.

In [ ]:
# Calculate min, max for color bar
vmin_y = single_glacier_raster.vy.mean(dim=["mid_date"]).min().data
vmax_y = single_glacier_raster.vy.mean(dim=["mid_date"]).max().data

vmin_x = single_glacier_raster.vx.mean(dim=["mid_date"]).min().data
vmax_x = single_glacier_raster.vx.mean(dim=["mid_date"]).max().data

vmin = min([vmin_x, vmin_y])
vmax = max([vmax_x, vmax_y])

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(17, 7))

x = single_glacier_raster.vx.mean(dim="mid_date").plot(ax=axs[0], vmin=vmin, vmax=vmax, cmap="RdBu_r")
y = single_glacier_raster.vy.mean(dim="mid_date").plot(ax=axs[1], vmin=vmin, vmax=vmax, cmap="RdBu_r")
axs[0].set_title("x-component velocity", fontsize=12)
axs[1].set_title("y-component velocity", fontsize=12)
fig.suptitle("Temporal mean of velocity components", fontsize=16, y=1.02)

x.colorbar.set_label("m/y", rotation=270)
y.colorbar.set_label("m/yr", rotation=270)

for i in range(len(axs)):
    axs[i].set_ylabel(None)
    axs[i].set_xlabel(None)
fig.supylabel("Y-coordinate of projection (meters)", x=0.08, fontsize=12)
fig.supxlabel("X-coordinate of projection (meters)", fontsize=12);

In addition to visualizing components (above), plotting velocity vectors is helpful for understanding magnitude and direction of flow:

First, calculate and visualize mean velocity magnitude over time (we will use the function defined in Part 1), and the mean estimated error over time:

In [ ]:
ds_v = calc_v_magnitude(single_glacier_raster.mean(dim="mid_date", skipna=True))

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20, 7))

single_glacier_vector.plot(ax=axs[0], facecolor="none", edgecolor="red")
single_glacier_raster.mean(dim="mid_date").plot.quiver("x", "y", "vx", "vy", ax=axs[1], angles="xy", robust=True)

single_glacier_vector.plot(ax=axs[1], facecolor="none", edgecolor="red")
a = ds_v["vmag"].plot(ax=axs[0], alpha=0.6, vmax=45, vmin=5)
a.colorbar.set_label("meter/year")

fig.supylabel("Y-coordinate of projection (meters)", x=0.08, fontsize=12)
fig.supxlabel("X-coordinate of projection (meters)", fontsize=12)

fig.suptitle(
    "Velocity vectors (R) and magntiude of velocity (L), averaged over time",
    fontsize=16,
    y=0.98,
)
for i in range(len(axs)):
    axs[i].set_xlabel(None)
    axs[i].set_ylabel(None)
    axs[i].set_title(None)

Visualize magnitude of velocity overlaid with velocity vectors next to velocity error:

In [ ]:
fig, ax = plt.subplots(figsize=(22, 6), ncols=2)

vmag = ds_v.vmag.plot(ax=ax[0], vmin=0, vmax=52, alpha=0.5)
single_glacier_raster.mean(dim="mid_date").plot.quiver("x", "y", "vx", "vy", ax=ax[0], angles="xy", robust=True)

err = ds_v.v_error.plot(ax=ax[1], vmin=0, vmax=52)


vmag.colorbar.set_label("m/y")  # , rotation=270)
err.colorbar.set_label("m/y")  # , rotation=270)

for i in range(len(ax)):
    ax[i].set_ylabel(None)
    ax[i].set_xlabel(None)
    ax[i].set_title(None)

fig.supxlabel("X-coordinate of projection (meters)", fontsize=12)
fig.supylabel("Y-coordinate of projection (meters)", x=0.08, fontsize=12)
fig.suptitle(
    "Mean velocity magnitude over time (L), mean error over time (R)",
    fontsize=16,
    y=1.02,
);

`v_error` is large relative to the magnitude of velocity, suggesting that this data is pretty noisy. 

### {{c3_its_nb4}}

Reduce over the spatial dimensions (this time we will switch it up and choose a different reduction function) and visualize variability over time:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

vmag_med = calc_v_magnitude(single_glacier_raster.median(dim=["x", "y"]))

vmag_med.plot.scatter(x="mid_date", y="vmag", ax=ax, marker="o", edgecolors="None", alpha=0.5)
fig.suptitle("Spatial median magnitude of velocity over time")
ax.set_title(None)
ax.set_ylabel("m/y")
ax.set_xlabel("Time");

This helps get a sense of velocity variability over time, but also shows how many outliers there are, even after taking the median over the x and y dimensions. In the final section of this notebook, we explore different approaches for changing the resolution of the temporal dimension.

## D. Computations  along time dimension

In Part 2, we saw that the timeseries is dense in places and quite noisy. This section demonstrates different approaches for looking at a temporal signal in the dataset.

### {{d1_its_nb4}}

Use Xarray's [`resample()`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.resample.html) method to coarsen the temporal resolution of the dataset. With `resample()`, you can upsample or downsample the data, choosing both the resample frequency and the type of reduction.

In [ ]:
# Coarsen dataset to monthly
resample_obj = single_glacier_raster.resample(mid_date="2ME")
# Calculate monthly median
glacier_resample_1mo = resample_obj.median(dim="mid_date")

The `mid_date` dimension is now much less dense:

In [ ]:
glacier_resample_1mo

Compare the 2-month resampled median time series to the full time series we plotted at the end of Part 2:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))

# Calculate magnitude of velocity after the temporal reduction
vmag_1mo = calc_v_magnitude(glacier_resample_1mo)

# Calculate spatial median
vmag_1mo["vmag"].median(dim=["x", "y"]).plot(ax=ax)
# Plot full time series (spatial median) magnitude of velocity
vmag_med.plot.scatter(
    x="mid_date",
    y="vmag",
    ax=ax,
    marker="o",
    edgecolors="None",
    alpha=0.5,
    color="orange",
)
# Labels and formatting
fig.suptitle(
    "2-month median magnitude of velocity (blue), full time series (orange)",
    fontsize=16,
)
ax.set_title(None)
ax.set_ylabel("m/y")
ax.set_xlabel("Time")
ax.set_ylim(0, 750);

We can make a few observations:
- Computing two-month median velocities makes it easier to see a somewhat periodic velocity signal
- As expected, the median is highly sensitive to the density of observations; as such, it displays greater amplitude during periods with sparser observations. When the density of observations increases significantly in 2014, the amplitude of variability of the 2-month median curve decreases dramatically. 

### {{d2_its_nb4}}

Xarray's [`groupby()`](https://docs.xarray.dev/en/stable/user-guide/groupby.html) functionality allows us to segment the dataset into different groups along given dimensions. Here, we use that to analyze seasonal velocity variability patterns:

In [ ]:
seasons_gb = single_glacier_raster.groupby(single_glacier_raster.mid_date.dt.season).median()
# add attrs to gb object
seasons_gb.attrs = single_glacier_raster.attrs
# Reorder seasons
seasons_gb = seasons_gb.reindex({"season": ["DJF", "MAM", "JJA", "SON"]})
seasons_gb

This is cool; we've gone from our 3-d object with a very dense `mid_date` dimension to a 3-d object where the temporal aspect of the data is represented by 4 seasons. 

In the above cell, we defined how we wanted to group our data (`single_glacier_raster.mid_date.dt.season`) and the reduction we wanted to apply to each group (`median()`). After the apply step, Xarray automatically combines the groups into a single object with a `season` dimension.

 If you'd like to see another example of this with more detailed explanations, go [here](https://tutorial.xarray.dev/fundamentals/03.2_groupby_with_xarray.html).

In [ ]:
# Calculate magnitude on seasonal groupby object
seasons_vmag = calc_v_magnitude(seasons_gb)

Use another `FacetGrid` plot to visualize the seasons side-by-side:

In [ ]:
fg = seasons_vmag.vmag.plot(col="season", cbar_kwargs={"label": "Meters / year"})
fg.fig.suptitle("Seasonal median velocity magnitude", fontsize=16, y=1.05)
fg.fig.supxlabel("X-coordinate of projection (meters)", fontsize=12, y=-0.05)
fg.fig.supylabel("Y-coordinate of projection (meters)", fontsize=12, x=-0.02)
for i in range(len(fg.axs[0])):
    fg.axs[0][i].set_ylabel(None)
    fg.axs[0][i].set_xlabel(None)
    fg.axs[0][i].tick_params(axis="x", labelrotation=45)

From the above FacetGrid plot, it appears that some regions of the glacier are very active (show high velocities) throughout the entire year. In other areas, it appears that glacier flow may be much more seasonal.

{{conclusion}}  
This was a primer in exploratory data analysis at the scale of an individual spatial area of interest (in this case, a glacier). The last notebook in this chapter will demonstrate exploratory analysis at a larger spatial scale.